# Seed 생성 확인용 테스트 노트북
http://4.230.26.44:8888

In [ ]:
import os
import sys
sys.path.append("/app")
from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F
os.environ["PYSPARK_SUBMIT_ARGS"] = """
    --conf spark.cores.max=4
    --conf spark.executor.memory=8g
    --conf spark.app.name=GDELT_Seed_Test
    pyspark-shell
"""

# Spark 세션 생성
spark = get_spark_session("GDELT_Seed_Test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

In [12]:
# Spark Metastore에 존재하는 모든 데이터베이스(스키마) 목록을 확인합니다.
print("📖 사용 가능한 데이터베이스(스키마) 목록:")
spark.sql("SHOW DATABASES").show()

📖 사용 가능한 데이터베이스(스키마) 목록:
+----------------+
|       namespace|
+----------------+
|         default|
|        gold_dev|
|       gold_prod|
|intermediate_dev|
|        seed_dev|
|       seed_prod|
|          silver|
|     staging_dev|
|    staging_prod|
+----------------+



In [13]:
# dbt seed로 생성된 테이블 확인
print("📜 'seed_dev' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN seed_dev").show()

📜 'seed_dev' 스키마의 테이블 목록:
+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| seed_dev|actor_ethnic_grou...|      false|
| seed_dev|actor_organizatio...|      false|
| seed_dev|actor_religion_codes|      false|
| seed_dev|    actor_role_codes|      false|
| seed_dev|  event_detail_codes|      false|
| seed_dev|event_quad_class_...|      false|
| seed_dev|    event_root_codes|      false|
| seed_dev|       geo_adm_codes|      false|
| seed_dev|   geo_country_codes|      false|
| seed_dev|      geo_type_codes|      false|
+---------+--------------------+-----------+



Seed 확인 및 관리

In [ ]:
# Seed 테이블(총 10개)의 샘플 데이터를 5개씩 출력하여 내용 확인
seed_tables = [
    "actor_ethnic_group_codes",
    "actor_organization_codes",
    "actor_religion_codes",
    "actor_role_codes",
    "event_root_codes",
    "event_detail_codes",
    "event_quad_class_codes",
    "geo_adm_codes",
    "geo_country_codes",
    "geo_type_codes",
]

for table in seed_tables:
    print(f"\n 🔍 {table} 테이블 샘플 데이터")
    try:
        spark.sql(f"SELECT * FROM seed_dev.{table} LIMIT 5").show(truncate=False)
    except Exception as e:
        print(f"❌ 테이블 조회 중, 오류 발생: {e}")

생성된 seed 테이블을 삭제할 경우, 아래 코드에 "삭제할 seed 테이블명"을 입력하여 실행

In [ ]:
# 삭제할 seed 테이블 이름 목록
seed_tables_to_drop = [
    # "[삭제할 seed 테이블명]"
    # "geo_country_fips_codes",
    # "geo_country_iso_codes"
]
print("🗑️ 기존 Seed 테이블 삭제를 시작합니다.")

for table_name in seed_tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS seed_dev.{table_name}")
        print(f"  - 테이블 'seed_dev.{table_name}' 삭제 완료.")
    except Exception as e:
        print(f"  - 테이블 'seed_dev.{table_name}' 삭제 중 오류 발생: {e}")

print("\n✅ 모든 Seed 테이블 삭제 작업이 완료되었습니다.")

# 삭제 후 테이블 목록을 다시 확인하여 깨끗해졌는지 확인
print("📜 현재 'seed_dev' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN seed_dev").show()

Staging 폴더의 SQL 파일을 dbt run한 결과 확인

In [14]:
# stg_seed_mapping 결과 확인
print("🥈 [stg_seed_mapping] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM staging_dev.stg_seed_mapping LIMIT 20").show(truncate=False)

🥈 [stg_seed_mapping] 뷰 : 상위 20개 데이터
+---------------+----------+-------------+-------------------+------------------------+----------+-------------+---------------+------------+-----------+------------+-----------------+-----------+-------------------+--------------------------+--------------------------------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+------------------------------------------------+-------------------------+--------------------------------------------------------+-------------------------+--------------+---------------+-----------+------------------+--------------------------+----------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+-----------------

In [15]:
# stg_seed_actors_parsed 결과 확인
print("🥈 [stg_seed_actors_parsed] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM staging_dev.stg_seed_actors_parsed LIMIT 20").show(truncate=False)

🥈 [stg_seed_actors_parsed] 뷰 : 상위 20개 데이터
+----------+-------------+-----+------------+-----+------------+-----+------------+-----+
|actor_code|description1 |type1|description2|type2|description3|type3|description4|type4|
+----------+-------------+-----+------------+-----+------------+-----+------------+-----+
|TURGOV    |튀르키예     |국가 |정부        |역할 |null        |null |null        |null |
|CANLAB    |캐나다       |국가 |노동        |역할 |null        |null |null        |null |
|PSE       |팔레스타인   |국가 |null        |null |null        |null |null        |null |
|ELI       |엘리트       |역할 |null        |null |null        |null |null        |null |
|PSEGOV    |팔레스타인   |국가 |정부        |역할 |null        |null |null        |null |
|USAJUD    |미국         |국가 |사법부      |역할 |null        |null |null        |null |
|HTI       |아이티       |국가 |null        |null |null        |null |null        |null |
|JAMGOV    |자메이카     |국가 |정부        |역할 |null        |null |null        |null |
|CHNGOVMIL |중국         |국가 |정부    

In [16]:
# stg_actors_description 결과 확인
print("🥈 [stg_actors_description] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM staging_dev.stg_actors_description LIMIT 20").show(truncate=False)

🥈 [stg_actors_description] 뷰 : 상위 20개 데이터


+---------------+------------------------------+-------------+
|global_event_id|actor1_info                   |actor2_info  |
+---------------+------------------------------+-------------+
|1264576819     |사법부                        |null         |
|1264576510     |영국                          |프랑스       |
|1264576155     |null                          |미국         |
|1264576136     |null                          |필리핀       |
|1264576325     |경찰                          |기업         |
|1264576901     |입법부                        |미국         |
|1264576443     |교육                          |보건         |
|1264576202     |null                          |알제리       |
|1264576378     |민간인                        |정부         |
|1264576388     |민간인                        |파키스탄 군대|
|1264577245     |미국                          |null         |
|1264577436     |미국                          |난민         |
|1264577488     |미국 민간인                   |미국         |
|1264576759     |이스라엘                      |팔레스타

In [18]:
# stg_gkg_detailed_events 결과 확인
print("🥈 [stg_gkg_detailed_events] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM staging_dev.stg_gkg_detailed_events LIMIT 20").show(truncate=False)

🥈 [stg_gkg_detailed_events] 뷰 : 상위 20개 데이터
+---------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------+----------+----------------+------------------+-------+-------------------+
|global_event_id|event_date|source_url                                                                                                                                                                     |mention_source_name|mention_doc_tone|v2_persons|v2_organizations|v2_enhanced_themes|amounts|processed_at       |
+---------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------+----------+----------------+------------------+-------+-------------------+
|12645

In [19]:
# int_fact_events_gkg 결과 확인
print("🥇 [int_fact_events_gkg] 뷰")
spark.sql("SELECT * FROM staging_dev.int_fact_events_gkg").show(truncate=False)

🥇 [int_fact_events_gkg] 뷰
+---------------+----------+-------------+-------------------+------------------------+----------+-------------+---------------+------------+-----------+------------+-----------------+-----------+-------------------+--------------------------+--------------------------------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+------------------------------------------------+-------------------------+--------------------------------------------------------+-------------------------+--------------+---------------+-----------+------------------+--------------------------+----------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+--------

Marts 폴더의 SQL 파일을 dbt run한 결과 확인

In [20]:
# gold_dashboard_master 결과 확인
print("🥇 [gold_dashboard_master] 뷰")
spark.sql("SELECT * FROM gold_dev.gold_dashboard_master").show(truncate=False)

🥇 [gold_dashboard_master] 뷰


+----------+--------------+--------------+--------------------------------------------------------+------------------+--------------+---------------+--------------+----------------------------+------------------+--------------+---------------+--------------+--------------------------+------------------+--------------+---------------+-----------+-------------------+-------------------+--------------+-------------+--------------+------------+------------+------------+------------+-----------------------+--------------------+--------------------+--------------------------+
|event_date|event_category|actor1_geo_iso|actor1_geo_eng                                          |actor1_geo_kor    |actor1_geo_lat|actor1_geo_long|actor2_geo_iso|actor2_geo_eng              |actor2_geo_kor    |actor2_geo_lat|actor2_geo_long|action_geo_iso|action_geo_eng            |action_geo_kor    |action_geo_lat|action_geo_long|event_count|avg_goldstein_scale|avg_tone           |total_mentions|total_sources|total_a

In [ ]:
# spark.stop()
print("\n✅ Spark 세션이 종료되었습니다.")